In [1]:
import os
import numpy as np
from keras import backend as K
from keras.models import Model
from keras.layers import concatenate as merge_l

from keras.optimizers import Adam, Nadam, Adadelta,SGD
from keras.layers import (
    Input, Convolution2D, MaxPooling2D, UpSampling2D,
    Reshape, core, Dropout, Flatten,
    Activation, BatchNormalization, Lambda, Dense, Conv2D, Conv2DTranspose, concatenate,Permute,Cropping2D,Add)
from keras.losses import binary_crossentropy
from keras.callbacks import (ModelCheckpoint, TensorBoard, CSVLogger, History, EarlyStopping, LambdaCallback,ReduceLROnPlateau)
from data_defores_generator import data_gen, create_list_id, split_black_image

Using TensorFlow backend.


In [2]:
def jaccard_coef(y_true, y_pred):
    smooth = 1e-12
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    smooth = 1e-12
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)

In [3]:
def unet(num_channel,size):
    conv_params = dict(activation='relu', border_mode='same')
    merge_params = dict(axis=-1)
    inputs1 = Input((size, size,int(num_channel)))
    conv1 = Convolution2D(32, (3,3), **conv_params)(inputs1)
    conv1 = Convolution2D(32, (3,3), **conv_params)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(64, (3,3), **conv_params)(pool1)
    conv2 = Convolution2D(64, (3,3), **conv_params)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(128, (3,3), **conv_params)(pool2)
    conv3 = Convolution2D(128, (3,3), **conv_params)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(256, (3,3), **conv_params)(pool3)
    conv4 = Convolution2D(256, (3,3), **conv_params)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(512, (3,3), **conv_params)(pool4)
    conv5 = Convolution2D(512, (3,3), **conv_params)(conv5)

    up6 = merge_l([UpSampling2D(size=(2, 2))(conv5), conv4], **merge_params)
    conv6 = Convolution2D(256, (3,3), **conv_params)(up6)
    conv6 = Convolution2D(256, (3,3), **conv_params)(conv6)

    up7 = merge_l([UpSampling2D(size=(2, 2))(conv6), conv3], **merge_params)
    conv7 = Convolution2D(128, (3,3), **conv_params)(up7)
    conv7 = Convolution2D(128, (3,3), **conv_params)(conv7)

    up8 = merge_l([UpSampling2D(size=(2, 2))(conv7), conv2], **merge_params)
    conv8 = Convolution2D(64, (3,3), **conv_params)(up8)
    conv8 = Convolution2D(64, (3,3), **conv_params)(conv8)

    up9 = merge_l([UpSampling2D(size=(2, 2))(conv8), conv1], **merge_params)
    conv9 = Convolution2D(32, (3,3), **conv_params)(up9)
    conv9 = Convolution2D(32, (3,3), **conv_params)(conv9)

    conv10 = Convolution2D(1, (1, 1), activation='sigmoid')(conv9)
    optimizer=SGD(lr=1e-4, decay=1e-8, momentum=0.9, nesterov=True)
    model = Model(input=inputs1, output=conv10)
    model.compile(optimizer=optimizer,
                loss=binary_crossentropy,
                metrics=['accuracy', jaccard_coef, jaccard_coef_int])
    return model

In [4]:

unet_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



C:\Users\SkyMap\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  """
C:\Users\SkyMap\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  
C:\Users\SkyMap\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  if __name__ == '__main__':
C:\Users\SkyMap\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\SkyMap\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 4)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 32) 1184        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 512, 512, 32) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 256, 256, 32) 0           conv2d_2[0][0]                  

In [5]:
from datetime import datetime
CURRENT_DATE = datetime.now().strftime("Ngay%dThang%mNam%Y_%Hh_%Mp_%Ss")
BATCH_SIZE=2
NUM_CHANNEL=4
NUM_CLASS=1
INPUT_SIZE=512
NUM_EPOCH = 500
EARLY=70
SPLIT_RATIO=0.9
MODEL_NAME="KGX_Unet_512_Nuoc"
IMAGE_PATH=r"C:\Users\SkyMap\Desktop\Test\Test\KGX_Unet_512_Nuoc\TrainingDataset\img_crop"
MASK_PATH=r"C:\Users\SkyMap\Desktop\Test\Test\KGX_Unet_512_Nuoc\TrainingDataset\img_mask_crop"
MODEL_DIR=r"C:\Users\SkyMap\Desktop\Test\Test\{}\{}".format(MODEL_NAME, CURRENT_DATE)
MODEL_DIR

'C:\\Users\\SkyMap\\Desktop\\Test\\Test\\KGX_Unet_512_Nuoc\\Ngay07Thang09Nam2021_14h_05p_14s'

In [24]:
unet_model = unet(NUM_CHANNEL, INPUT_SIZE)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
model_checkpoint = ModelCheckpoint(
        os.path.join(MODEL_DIR, "{}_val_weights.h5".format(CURRENT_DATE + "_{epoch:04d}")),
        verbose=1,
        save_best_only=True,
        save_weights_only=False)
model_earlystop = EarlyStopping(
        patience=EARLY,
        verbose=0,
        )
model_history = History()
model_board = TensorBoard(
    log_dir=os.path.join(MODEL_DIR, 'logs'),
    histogram_freq=0,
    write_graph=True,
    embeddings_freq=0)
lr_reducer = ReduceLROnPlateau(
    factor=np.sqrt(0.1),
    verbose=1,
    patience=10,
    min_lr=0.5e-7)
image_list= create_list_id(IMAGE_PATH)
np.random.shuffle(image_list)
count = len(image_list)    
cut_idx = int(round(count*SPLIT_RATIO))    
train_list = image_list[0:cut_idx]
val_list = [id_image for id_image in image_list if id_image not in train_list]

thres_neg = 1/100.0
scale_neg = 20/100.0
print("Train samples: {}".format(len(train_list)))
print("Validation samples: {}".format(len(val_list)))   
print("===========================")

pos_train,neg_train = split_black_image(image_list,MASK_PATH,thres_neg)
pos_val,neg_val = split_black_image(image_list,MASK_PATH,thres_neg)
step1 = round(len(pos_train)*(1+scale_neg)/BATCH_SIZE)
step2 = round(len(pos_val)*(1+scale_neg)/BATCH_SIZE)
num_chanel=NUM_CHANNEL
print(step2)
unet_model.fit_generator(
    generator= data_gen(pos_train,neg_train, IMAGE_PATH, MASK_PATH, BATCH_SIZE, scale_neg,num_chanel,NUM_CLASS,augment = True),
    validation_data = data_gen(pos_val,neg_val, IMAGE_PATH, MASK_PATH, BATCH_SIZE, scale_neg,num_chanel,NUM_CLASS),
    validation_steps = 100,
    steps_per_epoch = 500, 
    epochs=NUM_EPOCH, 
    verbose=1, 
    callbacks=[model_checkpoint,model_earlystop, model_history, model_board,lr_reducer])
unet_model.save_weights(os.path.join(MODEL_DIR, "{}_val_weights_last.h5".format(CURRENT_DATE)))

NameError: name 'unet' is not defined

In [25]:
INPUT_MOEL = 512
input_size = 512
crop_size = 400
num_chanel = 4




In [28]:
def write_window_many_chanel(output_ds, arr_c, s_h, e_h ,s_w, e_w, sw_w, sw_h, size_w_crop, size_h_crop):
    for c, arr in enumerate(arr_c):
        output_ds.write(arr[s_h:e_h,s_w:e_w],window = Window(sw_w, sw_h, size_w_crop, size_h_crop), indexes= c + 1)

In [29]:
def predict_small(cnn_model, img, dict_min_max=None):
    if dict_min_max:
        img = create_img_01(img, dict_min_max)
    img = img.swapaxes(0,1).swapaxes(1,2)
    img = cv2.resize(img,(INPUT_MOEL, INPUT_MOEL), interpolation = cv2.INTER_CUBIC)
    y_pre = cnn_model.predict(np.array([img/255]))
    y_pre = y_pre.reshape((INPUT_MOEL,INPUT_MOEL))
    y_pre =(y_pre>0.5).astype(np.uint8)
    y_pre = cv2.resize(y_pre,(input_size, input_size), interpolation = cv2.INTER_CUBIC)
    y_pre = np.array([y_pre])*255
    return y_pre

In [30]:
def predict_big(image_path, outputFileName1, dict_model):
    """
        Ham nay dung de predict 1 anh lon theo tung cua so truot.
        INPUT:
            image_path     : Duong dan anh can nhan dien
            outputFileName1: Duong dan file ket qua
            dict_model     : Chua ten class va duong dan model cua class.
    """
    head, tail  = os.path.split(outputFileName1)
    for key in dict_model:
        out_dir_name = os.path.join(head,key)
        if not os.path.exists(out_dir_name):
            os.makedirs(out_dir_name)
        outputFileName = os.path.join(out_dir_name, tail)
        fp_model = dict_model[key]

        # outputFileName = outputFileName[:-4] + f"_{key}.tif"
        cnn_model = unet_model(num_channel=num_chanel, size=INPUT_MOEL)
        cnn_model.load_weights(fp_model)

        # dict_min_max = get_min_max_image(image_path)
        dict_min_max = None
        with rasterio.open(image_path) as src:
            h,w = src.height,src.width
            source_crs = src.crs
            source_transform = src.transform
            # dtype_or = src.dtypes
            num_band = src.count

        # create img predict one band 
        with rasterio.open(outputFileName, 'w', driver='GTiff',
                                    height = h, width = w,
                                    count=1, dtype='uint8',
                                    crs=source_crs,
                                    transform=source_transform,
                                    nodata=0,
                                    compress='lzw') as output_ds:
            output_ds = np.empty((num_band,h,w))
            
        padding = int((input_size - crop_size)/2)
        list_weight = list(range(0, w, crop_size))
        list_hight = list(range(0, h, crop_size))

        src = rasterio.open(image_path)
        with rasterio.open(outputFileName,"r+") as output_ds:
            with tqdm(total=len(list_hight)*len(list_weight)) as pbar:
                for start_h_org in list_hight:
                    for start_w_org in list_weight:
                        # vi tri bat dau
                        h_crop_start = start_h_org - padding
                        w_crop_start = start_w_org - padding
                        # kich thuoc
                        tmp_img_size_model = np.zeros((num_band, input_size,input_size))
                        # truong hop 0 0
                        if h_crop_start < 0 and w_crop_start < 0:
                            # continue
                            h_crop_start = 0
                            w_crop_start = 0
                            size_h_crop = crop_size + padding
                            size_w_crop = crop_size + padding
                            img_window_crop  = src.read(window=Window(w_crop_start, h_crop_start, size_w_crop, size_h_crop))
                            tmp_img_size_model[:, padding:, padding:] = img_window_crop
                            img_predict = predict_small(cnn_model, tmp_img_size_model, dict_min_max)
                            write_window_many_chanel(output_ds, img_predict, padding, crop_size + padding, padding, 
                                                     crop_size + padding, start_w_org, start_h_org, crop_size, crop_size)
                        
                        # truong hop h = 0 va w != 0
                        elif h_crop_start < 0:
                            h_crop_start = 0
                            size_h_crop = crop_size + padding
                            size_w_crop = min(crop_size + 2*padding, w - start_w_org + padding)
                            img_window_crop  = src.read(window=Window(w_crop_start, h_crop_start, size_w_crop, size_h_crop))
                            
                            if size_w_crop == w - start_w_org + padding:
                                end_c_index_w =  size_w_crop
                                tmp_img_size_model[:,padding:,:end_c_index_w] = img_window_crop
                            else:
                                end_c_index_w = crop_size + padding
                                tmp_img_size_model[:, padding:,:] = img_window_crop
                            img_predict = predict_small(cnn_model, tmp_img_size_model, dict_min_max)
                            write_window_many_chanel(output_ds, img_predict, padding, crop_size + padding ,padding, 
                                                     end_c_index_w, start_w_org, start_h_org,  
                                                     min(crop_size, w - start_w_org), crop_size)
                        
                        # Truong hop w = 0, h!=0 
                        elif w_crop_start < 0:
                            w_crop_start = 0
                            size_w_crop = crop_size + padding
                            size_h_crop = min(crop_size + 2*padding, h - start_h_org + padding)
                            img_window_crop  = src.read(window=Window(w_crop_start, h_crop_start, size_w_crop, size_h_crop))
                            
                            if size_h_crop == h - start_h_org + padding:
                                end_c_index_h =  size_h_crop
                                tmp_img_size_model[:,:end_c_index_h,padding:] = img_window_crop
                            else:
                                end_c_index_h = crop_size + padding
                                tmp_img_size_model[:,:, padding:] = img_window_crop
                            img_predict = predict_small(cnn_model, tmp_img_size_model, dict_min_max)
                            write_window_many_chanel(output_ds, img_predict, padding, end_c_index_h, padding,crop_size + padding
                                                     , start_w_org, start_h_org, crop_size, min(crop_size, h - start_h_org))
                            
                        # Truong hop ca 2 deu khac khong
                        else:
                            size_w_crop = min(crop_size +2*padding, w - start_w_org + padding)
                            size_h_crop = min(crop_size +2*padding, h - start_h_org + padding)
                            img_window_crop  = src.read(window=Window(w_crop_start, h_crop_start, size_w_crop, size_h_crop))
                            # print(img_window_crop.shape, size_w_crop, size_h_crop)
                            if size_w_crop < (crop_size + 2*padding) and size_h_crop < (crop_size + 2*padding):
                                # print(img_window_crop.shape, size_w_crop, size_h_crop)
                                end_c_index_h = size_h_crop
                                end_c_index_w = size_w_crop
                                tmp_img_size_model[:,:end_c_index_h,:   end_c_index_w] = img_window_crop
                            elif size_w_crop < (crop_size + 2*padding):
                                end_c_index_h = crop_size + padding
                                end_c_index_w = size_w_crop
                                tmp_img_size_model[:,:,:end_c_index_w] = img_window_crop
                            elif size_h_crop < (crop_size + 2*padding):
                                end_c_index_w = crop_size + padding
                                end_c_index_h = size_h_crop
                                tmp_img_size_model[:,:end_c_index_h,:] = img_window_crop
                            else:
                                end_c_index_w = crop_size + padding
                                end_c_index_h = crop_size + padding
                                tmp_img_size_model[:,:,:] = img_window_crop
                            img_predict = predict_small(cnn_model, tmp_img_size_model, dict_min_max) 
                            write_window_many_chanel(output_ds, img_predict, padding, end_c_index_h, padding, end_c_index_w, 
                                                     start_w_org, start_h_org, 
                                                     min(crop_size, w - start_w_org), min(crop_size, h - start_h_org))
                        pbar.update()


In [31]:
 dict_fp_model = {
        "Nuoc":r"C:\Users\SkyMap\Desktop\Test\Test\KGX_Unet_512_Nuoc\Ngay07Thang09Nam2021_13h_14p_11.h5",
        "CoBui":r"C:\Users\SkyMap\Desktop\Test\Test\KGX_Unet_512_CoBui\Ngay08Thang09Nam2021_1h_07p_02.h5",
        "CoTan":r"C:\Users\SkyMap\Desktop\Test\Test\KGX_Unet_512_CoTan\Ngay09Thang09Nam2021_5h_45p_33.h5"
        }